In [1]:
import pandas as pd

In [2]:
game_details = pd.read_csv('games_details.csv')
game_details

/srv/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


GAME_ID     TEAM_ID TEAM_ABBREVIATION    TEAM_CITY  PLAYER_ID  \
0       22200477  1610612759               SAS  San Antonio    1629641   
1       22200477  1610612759               SAS  San Antonio    1631110   
2       22200477  1610612759               SAS  San Antonio    1627751   
3       22200477  1610612759               SAS  San Antonio    1630170   
4       22200477  1610612759               SAS  San Antonio    1630200   
...          ...         ...               ...          ...        ...   
668623  11200005  1610612743               DEN       Denver     202706   
668624  11200005  1610612743               DEN       Denver     202702   
668625  11200005  1610612743               DEN       Denver     201585   
668626  11200005  1610612743               DEN       Denver     202389   
668627  11200005  1610612743               DEN       Denver     201951   

            PLAYER_NAME NICKNAME START_POSITION COMMENT    MIN  ...  OREB  \
0        Romeo Langford    Romeo              F     NaN  18:06  ...   1.0   
1         Jeremy Sochan   Jeremy              F     NaN  31:01  ...   6.0   
2          Jakob Poeltl    Jakob              C     NaN  21:42  ...   1.0   
3         Devin Vassell    Devin              G     NaN  30:20  ...   0.0   
4             Tre Jones      Tre              G     NaN  27:44  ...   0.0   
...                 ...      ...            ...     ...    ...  ...   ...   
668623  Jordan Hamilton      NaN            NaN     NaN     19  ...   0.0   
668624   Kenneth Faried      NaN            NaN     NaN     23  ...   1.0   
668625     Kosta Koufos      NaN            NaN     NaN     15  ...   3.0   
668626   Timofey Mozgov      NaN            NaN     NaN     19  ...   1.0   
668627        Ty Lawson      NaN            NaN     NaN     27  ...   0.0   

        DREB  REB  AST  STL  BLK   TO   PF   PTS  PLUS_MINUS  
0        1.0  2.0  0.0  1.0  0.0  2.0  5.0   2.0        -2.0  
1        3.0  9.0  6.0  1.0  0.0  2.0  1.0  23.0       -14.0  
2        3.0  4.0  1.0  1.0  0.0  2.0  4.0  13.0        -4.0  
3        9.0  9.0  5.0  3.0  0.0  2.0  1.0  10.0       -18.0  
4        2.0  2.0  3.0  0.0  0.0  2.0  2.0  19.0         0.0  
...      ...  ...  ...  ...  ...  ...  ...   ...         ...  
668623   2.0  2.0  0.0  2.0  0.0  1.0  3.0  17.0         NaN  
668624   0.0  1.0  1.0  1.0  0.0  3.0  3.0  18.0         NaN  
668625   5.0  8.0  0.0  1.0  0.0  0.0  3.0   6.0         NaN  
668626   2.0  3.0  1.0  0.0  0.0  4.0  2.0   2.0         NaN  
668627   2.0  2.0  6.0  2.0  0.0  6.0  1.0   8.0         NaN  

[668628 rows x 29 columns]

In [3]:
game_details = game_details[['GAME_ID', 'TEAM_CITY', 'MIN', 'PTS']]
game_details['MIN'] = pd.to_numeric(game_details['MIN'], errors='coerce')
game_details['PTS'] = pd.to_numeric(game_details['PTS'], errors='coerce')
game_details = game_details.dropna(subset=['MIN', 'PTS'])
game_details

/tmp/ipykernel_1961/1375953088.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_details['MIN'] = pd.to_numeric(game_details['MIN'], errors='coerce')
/tmp/ipykernel_1961/1375953088.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_details['PTS'] = pd.to_numeric(game_details['PTS'], errors='coerce')


GAME_ID    TEAM_CITY   MIN   PTS
138383  10800114  San Antonio  19.0   3.0
138384  10800114  San Antonio  20.0   2.0
138385  10800114  San Antonio  23.0   9.0
138386  10800114  San Antonio  21.0  11.0
138387  10800114  San Antonio  16.0   6.0
...          ...          ...   ...   ...
668623  11200005       Denver  19.0  17.0
668624  11200005       Denver  23.0  18.0
668625  11200005       Denver  15.0   6.0
668626  11200005       Denver  19.0   2.0
668627  11200005       Denver  27.0   8.0

[23661 rows x 4 columns]

In [6]:
import statsmodels.api as sm
import pandas as pd
from statsmodels.stats.multitest import multipletests


X = sm.add_constant(game_details['MIN'])


y = game_details['PTS']

#regression model
model = sm.OLS(y, X).fit()

model_summary = model.summary()
p_values = model.pvalues

# Apply Bonferroni correction for multiple tests to control FWER
bonferroni_results = multipletests(p_values, alpha=0.05, method='bonferroni')

# Apply Benjamini-Hochberg correction for multiple tests to control FDR
bh_results = multipletests(p_values, alpha=0.05, method='fdr_bh')


summary_df = pd.DataFrame({
    'coef': model.params,
    'std_err': model.bse,
    't': model.tvalues,
    'p_value': model.pvalues,
    'bonferroni_p_value': bonferroni_results[1],
    'bh_p_value': bh_results[1],
    'bonferroni_reject': bonferroni_results[0],
    'bh_reject': bh_results[0]
})


bonferroni_discoveries = summary_df['bonferroni_reject'].sum()
bh_discoveries = summary_df['bh_reject'].sum()


print(f"Discoveries with FWER control (Bonferroni): {bonferroni_discoveries}")
print(f"Discoveries with FDR control (Benjamini-Hochberg): {bh_discoveries}")
print(model_summary)

Discoveries with FWER control (Bonferroni): 2
Discoveries with FDR control (Benjamini-Hochberg): 2
                            OLS Regression Results                            
Dep. Variable:                    PTS   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                 1.316e+04
Date:                Mon, 11 Dec 2023   Prob (F-statistic):               0.00
Time:                        19:48:44   Log-Likelihood:                -71040.
No. Observations:               23661   AIC:                         1.421e+05
Df Residuals:                   23659   BIC:                         1.421e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

In [7]:
import statsmodels.api as sm
import pandas as pd
from statsmodels.stats.multitest import multipletests

p_values = model.pvalues

bonferroni_results = multipletests(p_values, alpha=0.05, method='bonferroni')

bh_results = multipletests(p_values, alpha=0.05, method='fdr_bh')


summary_df = pd.DataFrame({
    'coef': model.params,
    'std_err': model.bse,
    't': model.tvalues,
    'p_value': model.pvalues,
    'bonferroni_p_value': bonferroni_results[1],
    'bh_p_value': bh_results[1],
    'bonferroni_reject': bonferroni_results[0],
    'bh_reject': bh_results[0]
})


bonferroni_discoveries = summary_df['bonferroni_reject'].sum()
bh_discoveries = summary_df['bh_reject'].sum()


print(f"Discoveries with FWER control (Bonferroni): {bonferroni_discoveries}")
print(f"Discoveries with FDR control (Benjamini-Hochberg): {bh_discoveries}")

Discoveries with FWER control (Bonferroni): 2
Discoveries with FDR control (Benjamini-Hochberg): 2


In [13]:
import statsmodels.api as sm
import pandas as pd
from statsmodels.stats.multitest import multipletests

team_city_dummies = pd.get_dummies(game_details['TEAM_CITY'], drop_first=True)


X = pd.concat([game_details['MIN'], team_city_dummies], axis=1)


X = sm.add_constant(X)


y = game_details['PTS']

modelx = sm.OLS(y, X).fit()
pvals = modelx.pvalues

# Control FWER using Bonferroni correction
bonferroni_adjusted_pvals = multipletests(pvals, alpha=0.05, method='bonferroni')

# Control FDR using Benjamini-Hochberg procedure
bh_adjusted_pvals = multipletests(pvals, alpha=0.05, method='fdr_bh')


summary_frame = pd.DataFrame({
    'coef': modelx.params.values,
    'std err': modelx.bse.values,
    't': modelx.tvalues.values,
    'P>|t|': modelx.pvalues.values,
    'Bonferroni adj P>|t|': bonferroni_adjusted_pvals[1],
    'FDR adj P>|t|': bh_adjusted_pvals[1],
    'Bonferroni reject': bonferroni_adjusted_pvals[0],
    'FDR reject': bh_adjusted_pvals[0]
}, index=modelx.params.index)

summary_frame
bonferroni_discoveries = summary_frame['Bonferroni reject'].sum()
fdr_discoveries = summary_frame['FDR reject'].sum()

print(f"Number of discoveries controlling FWER (Bonferroni): {bonferroni_discoveries}")
print(f"Number of discoveries controlling FDR (Benjamini-Hochberg): {fdr_discoveries}")
print(modelx.summary())

Number of discoveries controlling FWER (Bonferroni): 2
Number of discoveries controlling FDR (Benjamini-Hochberg): 2
                            OLS Regression Results                            
Dep. Variable:                    PTS   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.359
Method:                 Least Squares   F-statistic:                     428.7
Date:                Mon, 11 Dec 2023   Prob (F-statistic):               0.00
Time:                        19:51:25   Log-Likelihood:                -70994.
No. Observations:               23661   AIC:                         1.421e+05
Df Residuals:                   23629   BIC:                         1.423e+05
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
-----------

In [12]:
import statsmodels.api as sm
import pandas as pd
from statsmodels.stats.multitest import multipletests

pvals = modelx.pvalues

# Control FWER using Bonferroni correction
bonferroni_adjusted_pvals = multipletests(pvals, alpha=0.05, method='bonferroni')

# Control FDR using Benjamini-Hochberg procedure
bh_adjusted_pvals = multipletests(pvals, alpha=0.05, method='fdr_bh')


summary_frame = pd.DataFrame({
    'coef': modelx.params.values,
    'std err': modelx.bse.values,
    't': modelx.tvalues.values,
    'P>|t|': modelx.pvalues.values,
    'Bonferroni adj P>|t|': bonferroni_adjusted_pvals[1],
    'FDR adj P>|t|': bh_adjusted_pvals[1],
    'Bonferroni reject': bonferroni_adjusted_pvals[0],
    'FDR reject': bh_adjusted_pvals[0]
}, index=modelx.params.index)

summary_frame
bonferroni_discoveries = summary_frame['Bonferroni reject'].sum()
fdr_discoveries = summary_frame['FDR reject'].sum()

print(f"Number of discoveries controlling FWER (Bonferroni): {bonferroni_discoveries}")
print(f"Number of discoveries controlling FDR (Benjamini-Hochberg): {fdr_discoveries}")

Number of discoveries controlling FWER (Bonferroni): 2
Number of discoveries controlling FDR (Benjamini-Hochberg): 2


In [14]:
import statsmodels.api as sm
import pandas as pd
from statsmodels.stats.multitest import multipletests

game_details['MIN'] = pd.to_numeric(game_details['MIN'], errors='coerce')
game_details['PTS'] = pd.to_numeric(game_details['PTS'], errors='coerce')
game_details = game_details.dropna(subset=['MIN', 'PTS'])

team_city_dummies = pd.get_dummies(game_details['TEAM_CITY'], drop_first=True)

interaction_terms = pd.DataFrame(index=game_details.index)

for city in team_city_dummies.columns:
    interaction_terms[f"MIN_x_{city}"] = game_details['MIN'] * team_city_dummies[city]

X = pd.concat([game_details[['MIN']], team_city_dummies, interaction_terms], axis=1)

X = sm.add_constant(X)

y = game_details['PTS']

#regression model
model = sm.OLS(y, X).fit()
pvals = model.pvalues

bonferroni_adjusted_pvals = multipletests(pvals, alpha=0.05, method='bonferroni')

bh_adjusted_pvals = multipletests(pvals, alpha=0.05, method='fdr_bh')

summary_frame = pd.DataFrame({
    'coef': model.params.values,
    'std err': model.bse.values,
    't': model.tvalues.values,
    'P>|t|': model.pvalues.values,
    'Bonferroni adj P>|t|': bonferroni_adjusted_pvals[1],
    'FDR adj P>|t|': bh_adjusted_pvals[1],
    'Bonferroni reject': bonferroni_adjusted_pvals[0],
    'FDR reject': bh_adjusted_pvals[0]
}, index=model.params.index)

summary_frame
bonferroni_discoveries = summary_frame['Bonferroni reject'].sum()
fdr_discoveries = summary_frame['FDR reject'].sum()

print(f"Number of discoveries controlling FWER (Bonferroni): {bonferroni_discoveries}")
print(f"Number of discoveries controlling FDR (Benjamini-Hochberg): {fdr_discoveries}")
print(model.summary())

Number of discoveries controlling FWER (Bonferroni): 7
Number of discoveries controlling FDR (Benjamini-Hochberg): 18
                            OLS Regression Results                            
Dep. Variable:                    PTS   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     220.0
Date:                Mon, 11 Dec 2023   Prob (F-statistic):               0.00
Time:                        19:58:57   Log-Likelihood:                -70947.
No. Observations:               23661   AIC:                         1.420e+05
Df Residuals:                   23599   BIC:                         1.425e+05
Df Model:                          61                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----

In [11]:
from statsmodels.stats.multitest import multipletests

pvals = model.pvalues

bonferroni_adjusted_pvals = multipletests(pvals, alpha=0.05, method='bonferroni')

bh_adjusted_pvals = multipletests(pvals, alpha=0.05, method='fdr_bh')

summary_frame = pd.DataFrame({
    'coef': model.params.values,
    'std err': model.bse.values,
    't': model.tvalues.values,
    'P>|t|': model.pvalues.values,
    'Bonferroni adj P>|t|': bonferroni_adjusted_pvals[1],
    'FDR adj P>|t|': bh_adjusted_pvals[1],
    'Bonferroni reject': bonferroni_adjusted_pvals[0],
    'FDR reject': bh_adjusted_pvals[0]
}, index=model.params.index)

summary_frame
bonferroni_discoveries = summary_frame['Bonferroni reject'].sum()
fdr_discoveries = summary_frame['FDR reject'].sum()

print(f"Number of discoveries controlling FWER (Bonferroni): {bonferroni_discoveries}")
print(f"Number of discoveries controlling FDR (Benjamini-Hochberg): {fdr_discoveries}")

Number of discoveries controlling FWER (Bonferroni): 7
Number of discoveries controlling FDR (Benjamini-Hochberg): 18
